In [1]:
import pathlib

model_data_path = pathlib.Path("model_data").resolve()
gurobi_params = {}

In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import gurobipy_pandas as gppd

In [3]:
feasible_assignments = pd.read_feather(model_data_path / "feasible_assignments.feather")
staff_required = pd.read_feather(model_data_path / "staff_required.feather")
shift_conflicts = pd.read_feather(model_data_path / "shift_conflicts.feather")

In [4]:
with gp.Env(params=gurobi_params) as env, gp.Model(env=env) as model:

    model.ModelSense = GRB.MAXIMIZE
    assign = gppd.add_vars(
        model,
        feasible_assignments.set_index(["staff_id", "shift_id"]),
        obj=1.0,
        vtype=GRB.BINARY,
        name="assign",
    )
    
    constr_requires = gppd.add_constrs(
        model,
        assign.groupby("shift_id").sum(),
        GRB.LESS_EQUAL,
        staff_required["staff_count"],
        name="staffing",
    )

    df_conflict_vars = (
        shift_conflicts
        .join(assign.rename("assign1"), on=["staff_id", "shift1_id"])
        .join(assign.rename("assign2"), on=["staff_id", "shift2_id"])
        .dropna()
    )

    constr_conflicts = df_conflict_vars.gppd.add_constrs(
        model,
        "assign1 + assign2 <= 1",
        name="conflict",
    )

    model.optimize()

    assignments = assign.gppd.X
    assignments = assignments[assignments == 1.0].reset_index().drop(columns=["assign"])

assignments.to_feather(model_data_path / "assignments.feather")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 102 rows, 148 columns and 296 nonzeros
Model fingerprint: 0x6298e7f8
Variable types: 0 continuous, 148 integer (148 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 58.0000000
Presolve removed 93 rows and 134 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 11 (of 11 available processors)

Solution count 1: 58 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.800000000000e+01, be

In [5]:
assignments.sample(5)

,staff_id,shift_id
47,60131,16
35,60129,9
12,60127,6
53,60132,8
1,60126,22
